#Connect to drive

In [1]:
!git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi
%cd RecSys_Course_AT_PoliMi
!pip install -r requirements.txt
!python run_compile_all_cython.py

Cloning into 'RecSys_Course_AT_PoliMi'...
remote: Enumerating objects: 1493, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 1493 (delta 110), reused 225 (delta 97), pack-reused 1255
Receiving objects: 100% (1493/1493), 50.31 MiB | 25.63 MiB/s, done.
Resolving deltas: 100% (841/841), done.
/content/RecSys_Course_AT_PoliMi
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.0/978.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 kB 28.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies ... error
error: subprocess-exited-with-error

× pip subprocess t

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


#Import Library

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from typing import Tuple, Callable, Dict, Optional, List
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from scipy.stats import loguniform
!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00


#Load Data

In [4]:
%cd /gdrive/MyDrive/RECSYS
data_train = pd.read_csv('data_train.csv')
data_target_users_test = pd.read_csv('data_target_users_test.csv')
%cd RecSys_Course_AT_PoliMi

/gdrive/MyDrive/RECSYS


In [5]:
data_train.columns = ["user_id", "item_id", "Interaction"]

In [6]:
print ("The number of interactions is {}".format(len(data_train)))

The number of interactions is 478730


In [7]:
userID_unique = data_train["user_id"].unique()
itemID_unique = data_train["item_id"].unique()

In [8]:
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(data_train)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))

Number of items	 22222, Number of users	 12638
Max ID items	 22347, Max Id users	 13024



#Splitting Data

In [9]:
def preprocess_data(interactions: pd.DataFrame):
    unique_users = interactions.user_id.unique()
    unique_items = interactions.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    interactions = pd.merge(left=interactions,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    interactions = pd.merge(left=interactions,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return interactions


In [17]:
interactions = preprocess_data(data_train)

12638 1 13024
22222 1 22347


In [11]:
interactions

,user_id,item_id,Interaction,mapped_user_id,mapped_item_id
0,1,7,1.0,0,0
1,2,7,1.0,1,0
2,26,7,1.0,24,0
3,36,7,1.0,34,0
4,41,7,1.0,39,0
...,...,...,...,...,...
478725,12962,20368,1.0,12579,22217
478726,12985,21058,1.0,12601,22218
478727,12989,22317,1.0,12605,22219
478728,13009,22339,1.0,12624,22220


In [18]:
def dataset_splits(interactions, num_users, num_items, validation_percentage: float, testing_percentage: float):
    seed = 1234

    (user_ids_training, user_ids_test,
     item_ids_training, item_ids_test,
     interactions_training, interactions_test) = train_test_split(interactions.mapped_user_id,
                                                        interactions.mapped_item_id,
                                                        interactions.Interaction,
                                                        test_size=testing_percentage,
                                                        shuffle=True,
                                                        random_state=seed)

    (user_ids_training, user_ids_validation,
     item_ids_training, item_ids_validation,
     interactions_training, interactions_validation) = train_test_split(user_ids_training,
                                                              item_ids_training,
                                                              interactions_training,
                                                              test_size=validation_percentage,
                                                             )

    urm_train = sp.csr_matrix((interactions_training, (user_ids_training, item_ids_training)),
                              shape=(num_users, num_items))

    urm_validation = sp.csr_matrix((interactions_validation, (user_ids_validation, item_ids_validation)),
                              shape=(num_users, num_items))

    urm_test = sp.csr_matrix((interactions_test, (user_ids_test, item_ids_test)),
                              shape=(num_users, num_items))



    return urm_train, urm_validation, urm_test



In [13]:
urm_train, urm_validation, urm_test = dataset_splits(interactions,
                                                     num_users=12638,
                                                     num_items=22222,
                                                     validation_percentage=0.15,
                                                     testing_percentage=0.15)

In [14]:
urm_train

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 345882 stored elements in Compressed Sparse Row format>

In [15]:
urm_validation

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 61038 stored elements in Compressed Sparse Row format>

In [16]:
urm_test

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 71810 stored elements in Compressed Sparse Row format>

#Build the Recommendation System

In [ ]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [ ]:
itemKNNCF = ItemKNNCFRecommender(urm_train)
itemKNNCF.fit(topK = optuna_trial.suggest_int("topK", 5, 1000),
              shrink = optuna_trial.suggest_int("shrink", 0, 1000),
              similarity =  optuna_trial.suggest_categorical("similarity", ["cosine", "jaccard", "adjusted", "asymmetric","pearson",  "dice" , "tversky",  "tanimoto"]),
              normalize = optuna_trial.suggest_categorical("normalize", [True, False]))

result_df, _ = evaluator_validation.evaluateRecommender(itemKNNCF)
'topK': 73, 'shrink': 36, 'similarity': 'tversky', 'normalize': True

## Top popular for the users without info

In [19]:
class TopPopRecommender(object):

    def fit(self, URM_train):

        item_popularity = np.ediff1d(URM_train.tocsc().indptr)

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = np.argsort(item_popularity)
        self.popular_items = np.flip(self.popular_items, axis = 0)


    def recommend(self, user_id, at=5):

        recommended_items = self.popular_items[0:at]

        return recommended_items

#Hyperparameter Tuning


*   Number of neighbors

*   Shrinking

*   Similarity Type



In [25]:
%cd RecSys_Course_AT_PoliMi_
from Evaluation.Evaluator import EvaluatorHoldout


evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

[Errno 2] No such file or directory: 'RecSys_Course_AT_PoliMi_'
/gdrive/MyDrive/RECSYS/RecSys_Course_AT_PoliMi
EvaluatorHoldout: Ignoring 3066 (24.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2716 (21.5%) Users that have less than 1 test interactions


In [26]:
def objective_function_ItemKNN_similarities(optuna_trial):

    recommender_instance = ItemKNNCFRecommender(urm_train)
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])

    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 5, 1000),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 1000),
                  }

    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])


    recommender_instance.fit(**full_hyperp)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [27]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

def objective_function_UserKNN_similarities(optuna_trial):

    recommender_instance = UserKNNCFRecommender(urm_train)
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean', 'pearson'])

    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 5, 1000),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 1000),
                  }

    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])


    recommender_instance.fit(**full_hyperp)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [28]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame()

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)

In [29]:
item_optuna_study = optuna.create_study(direction="maximize")

item_save_results = SaveResults()

item_optuna_study.optimize(objective_function_ItemKNN_similarities,
                      callbacks=[item_save_results],
                      n_trials = 50)

[I 2023-11-24 17:53:37,718] A new study created in memory with name: no-name-4bca15dc-f8a7-4103-b316-49eb9d1ff7ed


ItemKNNCFRecommender: URM Detected 336 ( 2.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 172 ( 0.8%) items with no interactions.
Similarity column 22222 (100.0%), 4188.67 column/sec. Elapsed time 5.31 sec
EvaluatorHoldout: Processed 9572 (100.0%) in 13.02 sec. Users per second: 735


[I 2023-11-24 17:53:56,388] Trial 0 finished with value: 0.01576983795328964 and parameters: {'similarity': 'cosine', 'topK': 345, 'shrink': 843}. Best is trial 0 with value: 0.01576983795328964.


ItemKNNCFRecommender: URM Detected 336 ( 2.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 172 ( 0.8%) items with no interactions.


<ipython-input-28-14c86811ef81>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


...

[I 2023-11-24 18:11:35,642] Trial 48 finished with value: 0.02393404373868229 and parameters: {'similarity': 'tversky', 'topK': 662, 'shrink': 51, 'tversky_alpha': 1.9684121165858137, 'tversky_beta': 1.5728505343748114}. Best is trial 43 with value: 0.023990255142313194.


ItemKNNCFRecommender: URM Detected 336 ( 2.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 172 ( 0.8%) items with no interactions.


<ipython-input-28-14c86811ef81>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


Similarity column 22222 (100.0%), 4520.19 column/sec. Elapsed time 4.92 sec
EvaluatorHoldout: Processed 9572 (100.0%) in 14.21 sec. Users per second: 674


[I 2023-11-24 18:11:55,342] Trial 49 finished with value: 0.021969049111495758 and parameters: {'similarity': 'tversky', 'topK': 692, 'shrink': 283, 'tversky_alpha': 1.9704040453112468, 'tversky_beta': 1.5502522844180822}. Best is trial 43 with value: 0.023990255142313194.
<ipython-input-28-14c86811ef81>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


In [30]:
user_optuna_study = optuna.create_study(direction="maximize")

user_save_results = SaveResults()

user_optuna_study.optimize(objective_function_UserKNN_similarities,
                      callbacks=[user_save_results],
                      n_trials = 50)

[I 2023-11-24 18:12:11,901] A new study created in memory with name: no-name-0d65c4c4-b00d-4608-8e8f-7393471d36af


UserKNNCFRecommender: URM Detected 336 ( 2.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 172 ( 0.8%) items with no interactions.
Similarity column 12638 (100.0%), 5728.65 column/sec. Elapsed time 2.21 sec
EvaluatorHoldout: Processed 9572 (100.0%) in 11.23 sec. Users per second: 852


[I 2023-11-24 18:12:25,591] Trial 0 finished with value: 0.018399801172732627 and parameters: {'similarity': 'asymmetric', 'topK': 605, 'shrink': 938, 'asymmetric_alpha': 0.25101267692936013}. Best is trial 0 with value: 0.018399801172732627.


UserKNNCFRecommender: URM Detected 336 ( 2.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 172 ( 0.8%) items with no interactions.


<ipython-input-28-14c86811ef81>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


...

[I 2023-11-24 18:22:40,499] Trial 48 finished with value: 0.01925273947160702 and parameters: {'similarity': 'tversky', 'topK': 303, 'shrink': 66, 'tversky_alpha': 0.9780514107618918, 'tversky_beta': 1.557706914474722}. Best is trial 39 with value: 0.019333136827652083.


UserKNNCFRecommender: URM Detected 336 ( 2.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 172 ( 0.8%) items with no interactions.


<ipython-input-28-14c86811ef81>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


Similarity column 12638 (100.0%), 7520.81 column/sec. Elapsed time 1.68 sec
EvaluatorHoldout: Processed 9572 (100.0%) in 6.60 sec. Users per second: 1451


[I 2023-11-24 18:22:49,578] Trial 49 finished with value: 1.5148349352277475e-05 and parameters: {'similarity': 'pearson', 'topK': 332, 'shrink': 70}. Best is trial 39 with value: 0.019333136827652083.
<ipython-input-28-14c86811ef81>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


In [31]:
user_bests = user_optuna_study.best_trial.params

In [32]:
item_bests = {'topK': 73, 'shrink': 36, 'similarity': 'tversky', 'normalize': True}

In [33]:
recommender_instance_item = ItemKNNCFRecommender(urm_train + urm_validation)
recommender_instance_item.fit(**item_bests)

result_df_item, _ = evaluator_test.evaluateRecommender(recommender_instance_item)
result_df_item

ItemKNNCFRecommender: URM Detected 156 ( 1.2%) users with no interactions.
ItemKNNCFRecommender: URM Detected 65 ( 0.3%) items with no interactions.
Similarity column 22222 (100.0%), 3688.07 column/sec. Elapsed time 6.03 sec
EvaluatorHoldout: Processed 9922 (100.0%) in 7.66 sec. Users per second: 1296


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.068807,0.14179,0.127972,0.033332,0.06822,0.214166,0.12179,0.089495,0.439428,0.265079,...,0.785093,0.344991,0.785093,0.04494,9.641491,0.996131,0.128607,0.742568,2.120294,0.26183


In [35]:
recommender_instance_user = UserKNNCFRecommender(urm_train + urm_validation)
recommender_instance_user.fit(**user_optuna_study.best_trial.params)

result_df_user, _ = evaluator_test.evaluateRecommender(recommender_instance_user)
result_df_user

UserKNNCFRecommender: URM Detected 156 ( 1.2%) users with no interactions.
UserKNNCFRecommender: URM Detected 65 ( 0.3%) items with no interactions.
Similarity column 12638 (100.0%), 1790.04 column/sec. Elapsed time 7.06 sec
EvaluatorHoldout: Processed 9922 (100.0%) in 9.77 sec. Users per second: 1016


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.055614,0.119083,0.108848,0.026084,0.056232,0.185723,0.101881,0.073615,0.388228,0.219411,...,0.785093,0.304795,0.785093,0.012461,7.632904,0.983943,0.035661,0.587871,3.616883,0.23348


In [36]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sp.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha = 0.5):
        self.alpha = alpha


    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [55]:
scoreshybridrecommender = ScoresHybridRecommender(urm_train, recommender_instance_item, recommender_instance_user)
scoreshybridrecommender.fit(alpha = 0.3)

result_df, _ = evaluator_validation.evaluateRecommender(scoreshybridrecommender)
result_df

ScoresHybridRecommender: URM Detected 336 ( 2.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 172 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 9572 (100.0%) in 13.23 sec. Users per second: 723


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.156644,0.445505,0.430197,0.118654,0.356519,0.68971,0.456927,0.229663,0.835144,0.901088,...,0.757398,0.632537,0.757398,0.015053,7.884771,0.985486,0.043457,0.607636,3.555524,0.276227


#Evaluate the Recommendation System

In [ ]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant) / relevant_items.shape[0]

    return recall_score


def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)

    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

In [ ]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = urm_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue

        recommendations = recommender.recommend(user_id=user_id,
                                               at=recommendation_length,
                                               urm_train=urm_train,
                                               remove_seen=True)

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemknn_recommender,
                                                                                            urm_train,
                                                                                            urm_test)

NameError: ignored

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

#Prepare Submission

In [39]:
#forse ho sbagliato ad agggiungere anche il test set qui
best_shrink = 5
urm_train_validation = urm_train + urm_validation + urm_test
urm_train_validation

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [45]:
best_recommender = ScoresHybridRecommender(urm_train_validation, recommender_instance_item, recommender_instance_user)
best_recommender.fit(alpha = 0.5)

##Top popular for those users without info

In [41]:
toppop_recommender = TopPopRecommender()
toppop_recommender.fit(urm_train_validation)

In [42]:
users_to_recommend = np.array(data_target_users_test["user_id"])
len(users_to_recommend)

10882

In [47]:
def prepare_submission(interactions: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object, toppop_recommender: object):
    users_ids_and_mappings = interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates()
    items_ids_and_mappings = interactions[["item_id", "mapped_item_id"]].drop_duplicates()

    mapping_to_item_id = dict(zip(interactions.mapped_item_id, interactions.item_id))


    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id

        recommendations = recommender.recommend(user_id_array=mapped_user_id, cutoff=10)

        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    #assign top popular to the user without interactions
    users_without_info = np.setdiff1d(users_to_recommend,np.array(interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates().user_id))
    for user_id in users_without_info:
      recommendations = toppop_recommender.recommend(user_id, at = 10)
      submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    return submission

In [48]:
submission = prepare_submission(interactions, users_to_recommend, urm_train_validation, best_recommender, toppop_recommender)

In [49]:
submission

[(1, [12943, 1671, 876, 13590, 2226, 1013, 3991, 776, 14708, 1676]),
 (2, [2, 6, 3, 4, 15, 5, 9, 18, 8, 19]),
 (26, [139, 254, 474, 165, 314, 1005, 269, 1089, 41, 350]),
 (36, [4, 6, 3, 14, 15, 19, 22, 44, 20, 74]),
 (41, [31, 55, 58, 4, 6, 1, 20, 9, 5, 8]),
 (47, [96, 38, 4, 15, 3, 2, 214, 85, 344, 72]),
 (54, [2, 4, 6, 8, 19, 1, 3, 30, 48, 11]),
 (73, [21, 3, 109, 112, 69, 81, 2, 158, 36, 247]),
 (88, [99, 133, 6, 4, 139, 346, 3, 318, 2, 1]),
 (89, [138, 324, 344, 10, 2, 59, 35, 1, 3, 186]),
 (95, [26, 6, 25, 112, 2, 14, 15, 339, 1, 44]),
 (100, [313, 99, 3, 764, 636, 15, 11, 10, 119, 6]),
 (101, [82, 837, 754, 343, 624, 282, 1207, 2314, 1177, 89]),
 (102, [2843, 2958, 2284, 2904, 2585, 2283, 3208, 2288, 1795, 2147]),
 (104, [2, 1, 4, 3, 10, 9, 6, 8, 11, 12]),
 (114, [5, 17, 27, 42, 50, 37, 2, 145, 23, 1]),
 (124, [34, 94, 346, 715, 1198, 3, 4, 8, 262, 1]),
 (127, [11, 42, 50, 47, 35, 2, 37, 41, 40, 14]),
 (138, [3, 117, 137, 120, 15, 2, 30, 10, 18, 54]),
 (145, [1, 56, 3, 6, 144, 28

In [50]:
def write_submission(submissions):
    with open("./submission.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [51]:
%cd /gdrive/MyDrive/RECSYS
write_submission(submission)

/gdrive/MyDrive/RECSYS
